In [200]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize, differential_evolution

## Notas personales

$$
CVaR_w (\alpha) = -E[r_w \ | \ r_w \leq -VaR_w (\alpha)]
$$


$$
C_{(i)}CVaR_w (\alpha) = -E[w_{(i)}r_{(i)} \ | \ r_w \leq -VaR_w (\alpha)]
$$


$$
\%C_{(i)}CVaR_w(\alpha) = \frac{E[w_{(i)}r_{(i)} \ | \ r_w \leq -VaR_w(\alpha)]}{E[r_w \ | \ r_w \leq -VaR_w(\alpha)]}
$$

MCC portfolio:

$$
w^{MCC} = argmin \ C_w (\alpha)
$$

$$
\ \ w \in W
$$

where

$$
C_w(\alpha) = max_i \ C_{(i)}CVaR_w (\alpha)
$$

The other two strategies are the minimum CVaR (MC) and equal-weight (EW) portfolios:

$$
w^{minCVaR} = argmin_{w \in W}CVaR_w (\alpha) \ \text{and} \ w^{EW} = (1/N, ... , 1/N)'
$$

## Download data

In [201]:
# bonds, commodities, equities and real estate
tickers = ['VBTLX', 'GSG', 'VTI', 'VNQ']
data = yf.download(tickers, start='2020-01-01')['Adj Close']

# calculate returns
returns = data.pct_change().dropna()

# defining alpha and the number of assets
alpha = 0.05
n_assets = len(returns.columns)

[*********************100%***********************]  4 of 4 completed


## Calculate CVaR

In [202]:
# CVaR for only long positions
def portfolio_return(weights):
    return returns.dot(weights)

# Better way to calculate CVaR than the one used in my homework 1. I used .query in the homework, but checking with friends this way is better.
def cvar(portfolio_returns, alpha):
    var = np.percentile(portfolio_returns, alpha*100)
    cvar = -portfolio_returns[portfolio_returns < var].mean()
    return cvar

def individual_cvar_contributions(weights, returns, alpha):
    portfolio_returns = portfolio_return(weights)
    var = np.percentile(portfolio_returns, alpha*100)

    # check which days are in the cvar for the portfolio
    bad_days_portfolio = portfolio_returns < var

    contributions = []
    # chech the returns of each asset the days where the portfolio is in the cvar to know the contribution
    for i in range(n_assets):
        asset_contribution = -returns.iloc[:, i][bad_days_portfolio].mean() * weights[i]
        contributions.append(asset_contribution)
    
    portfolio_cvar = cvar(portfolio_returns, alpha)

    percentage_contributions = []
    for j in range(len(contributions)):
        pct_contributions = contributions[j] / portfolio_cvar
        percentage_contributions.append(pct_contributions)
    
    return contributions

## Optimization

In [203]:
def optimal_mcc(weights, returns, alpha):
    cvar_contributions = individual_cvar_contributions(weights, returns, alpha)
    return np.max(cvar_contributions)

constraints = [
    {"type": "eq", "fun": lambda w: np.sum(w) - 1},
]
bounds = tuple((0, 1) for _ in range(n_assets))

# Initial guess
initial_weights = np.ones(n_assets) / n_assets

result = minimize(
    fun=optimal_mcc,
    x0=initial_weights,
    args=(returns, alpha),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
    tol=1e-8
)
mcc_weights = result.x



def min_cvar(weights, returns, alpha):
    portfolio_returns = portfolio_return(weights)
    return cvar(portfolio_returns, alpha)

result_min_cvar = minimize(
    fun=min_cvar,
    x0=initial_weights,
    args=(returns, alpha),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
    tol=1e-8
)
min_cvar_weights = result_min_cvar.x


## Results

In [204]:
mcc_cvar_contrubutions = individual_cvar_contributions(mcc_weights, returns, alpha)
mcc_cvar = cvar(portfolio_return(mcc_weights), alpha)
mcc_cvar_pct_contributions = mcc_cvar_contrubutions / mcc_cvar

print(f'MCC Portfolio weights:')
print('---------------------')
for i in range(n_assets):
    print(f'{returns.keys()[i]}: {mcc_weights[i]:.3%}')
print('---------------------')

print()

print(f'MCC Portfolio CVaR percentage contribution:')
print('---------------------')
for i in range(n_assets):
    print(f'{returns.keys()[i]}: {mcc_cvar_pct_contributions[i]:.3%}')
print('---------------------')


MCC Portfolio weights:
---------------------
GSG: 14.133%
VBTLX: 67.600%
VNQ: 6.835%
VTI: 11.432%
---------------------

MCC Portfolio CVaR percentage contribution:
---------------------
GSG: 31.120%
VBTLX: 31.351%
VNQ: 16.182%
VTI: 21.347%
---------------------


In [212]:
min_cvar_cvar_contrubutions = individual_cvar_contributions(min_cvar_weights, returns, alpha)
min_cvar_cvar = cvar(portfolio_return(min_cvar_weights), alpha)
min_cvar_cvar_pct_contributions = min_cvar_cvar_contrubutions / min_cvar_cvar

print(f'Min CVaR Portfolio weights:')
print('---------------------')
for i in range(n_assets):
    print(f'{returns.keys()[i]}: {min_cvar_weights[i]:.3%}')
print('---------------------')

print()

print(f'Min CVaR Portfolio CVaR percentage contribution:')
print('---------------------')
for i in range(n_assets):
    print(f'{returns.keys()[i]}: {min_cvar_cvar_pct_contributions[i]:.3%}')
print('---------------------')

Min CVaR Portfolio weights:
---------------------
Asset 1: 3.905%
Asset 2: 94.151%
Asset 3: 0.000%
Asset 4: 1.943%
---------------------

Min CVaR Portfolio CVaR percentage contribution:
---------------------
Asset 1: 0.064%
Asset 2: 99.864%
Asset 3: -0.000%
Asset 4: 0.072%
---------------------


In [206]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, differential_evolution

# ----------------------------
# Step 2: Define CVaR Functions
# ----------------------------
alpha = 0.05  # 95% CVaR

def compute_portfolio_returns(weights, returns):
    return returns.dot(weights)

def compute_cvar(portfolio_returns, alpha):
    var = np.quantile(portfolio_returns, alpha)
    cvar = -portfolio_returns[portfolio_returns <= var].mean()
    return cvar

def compute_cvar_contributions(weights, returns, alpha):
    portfolio_returns = compute_portfolio_returns(weights, returns)
    var = np.quantile(portfolio_returns, alpha)
    tail_scenarios = portfolio_returns <= var
    contributions = []
    for i in range(len(weights)):
        asset_contribution = -returns.iloc[:, i][tail_scenarios].mean() * weights[i]
        contributions.append(asset_contribution)
    total_cvar = compute_cvar(portfolio_returns, alpha)
    # Standardize contributions as a percentage of total CVaR
    contributions_pct = [c / total_cvar for c in contributions] if total_cvar != 0 else [0]*len(weights)
    return np.array(contributions_pct)

# ----------------------------
# Step 3: Optimization for MCC Portfolio
# ----------------------------
def objective_mcc(weights, returns, alpha):
    contributions = compute_cvar_contributions(weights, returns, alpha)
    return np.max(contributions)  # Minimize the largest CVaR contribution

# Constraints: sum(weights) = 1, weights >= 0
constraints = (
    {"type": "eq", "fun": lambda w: np.sum(w) - 1},
)
bounds = tuple((0, 1) for _ in range(n_assets))

# Initial guess (equal-weighted)
initial_weights = np.ones(n_assets) / n_assets

# Solve using Sequential Least Squares Programming (SLSQP)
result = minimize(
    fun=objective_mcc,
    x0=initial_weights,
    args=(returns, alpha),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
    tol=1e-8
)
mcc_weights = result.x

# ----------------------------
# Step 4: Compare with Other Strategies
# ----------------------------
# Equal-weighted portfolio
ew_weights = np.ones(n_assets) / n_assets

# Minimum CVaR portfolio (minimizes total CVaR)
def objective_min_cvar(weights, returns, alpha):
    portfolio_returns = compute_portfolio_returns(weights, returns)
    return compute_cvar(portfolio_returns, alpha)

result_min_cvar = minimize(
    fun=objective_min_cvar,
    x0=initial_weights,
    args=(returns, alpha),
    method="SLSQP",
    bounds=bounds,
    constraints=constraints,
    tol=1e-8
)
min_cvar_weights = result_min_cvar.x

# ----------------------------
# Step 5: Print Results
# ----------------------------
print("Optimal Weights:")
print(f"- MCC Portfolio: {np.round(mcc_weights, 3)}")
print(f"- Equal-Weighted: {np.round(ew_weights, 3)}")
print(f"- Minimum CVaR: {np.round(min_cvar_weights, 3)}")

# Compute CVaR contributions for each portfolio
def print_contributions(weights, name):
    contributions = compute_cvar_contributions(weights, returns, alpha)
    print(f"\n{name} Portfolio CVaR Contributions (%):")
    for i, asset in enumerate(returns.columns):
        print(f"{asset}: {contributions[i]:.1%}")

print_contributions(mcc_weights, "MCC")
print_contributions(ew_weights, "Equal-Weighted")
print_contributions(min_cvar_weights, "Minimum CVaR")

Optimal Weights:
- MCC Portfolio: [0.127 0.637 0.106 0.13 ]
- Equal-Weighted: [0.25 0.25 0.25 0.25]
- Minimum CVaR: [0.039 0.942 0.    0.019]

MCC Portfolio CVaR Contributions (%):
GSG: 25.1%
VBTLX: 25.1%
VNQ: 25.1%
VTI: 24.7%

Equal-Weighted Portfolio CVaR Contributions (%):
GSG: 32.0%
VBTLX: 1.2%
VNQ: 36.2%
VTI: 30.6%

Minimum CVaR Portfolio CVaR Contributions (%):
GSG: 3.9%
VBTLX: 94.1%
VNQ: 0.0%
VTI: 1.9%


In [207]:
import numpy as np
import pandas as pd

def calculate_portfolio_cvar(returns, weights, alpha=0.05):
    """
    Calculate the portfolio CVaR and each asset's contribution to CVaR.

    Parameters:
    - returns: DataFrame of asset returns (rows = time periods, columns = assets).
    - weights: Array of portfolio weights (1D numpy array, must sum to 1).
    - alpha: Confidence level for CVaR (default = 0.05 for 95% confidence).

    Returns:
    - portfolio_cvar: Portfolio CVaR (a negative number).
    - cvar_contributions: List of each asset's CVaR contribution.
    - percentage_contributions: Contribution of each asset as a percentage of total CVaR.
    """
    # Ensure weights sum to 1
    weights = weights / np.sum(weights)
    
    # Step 1: Compute portfolio returns
    portfolio_returns = returns.dot(weights)
    
    # Step 2: Calculate VaR threshold and identify tail scenarios
    var_threshold = np.quantile(portfolio_returns, alpha)
    tail_mask = portfolio_returns <= var_threshold  # Boolean mask for losses beyond VaR
    
    # Step 3: Calculate portfolio CVaR
    portfolio_cvar = -portfolio_returns[tail_mask].mean()  # Average of tail losses
    
    # Step 4: Calculate individual asset contributions to CVaR
    cvar_contributions = []
    for i in range(len(weights)):
        asset_returns = returns.iloc[:, i]
        avg_loss = -asset_returns[tail_mask].mean()  # Average loss of the asset in tail scenarios
        contribution = weights[i] * avg_loss  # Weighted contribution of the asset
        cvar_contributions.append(contribution)
    
    # Calculate percentage contributions
    percentage_contributions = [c / portfolio_cvar for c in cvar_contributions]
    
    return portfolio_cvar, cvar_contributions, percentage_contributions

# Example usage
if __name__ == "__main__":
    # Generate random data for 4 assets
    np.random.seed(42)
    returns = pd.DataFrame({
        'Asset 1': np.random.normal(0.001, 0.02, 1000),
        'Asset 2': np.random.normal(0.001, 0.03, 1000),
        'Asset 3': np.random.normal(0.002, 0.015, 1000),
        'Asset 4': np.random.normal(0.0005, 0.025, 1000),
    })
    
    # Portfolio weights
    weights = np.array([0.3, 0.4, 0.2, 0.1])  # Example weights (must sum to 1)
    
    # Calculate portfolio CVaR and contributions
    alpha = 0.05  # 95% confidence level
    portfolio_cvar, cvar_contributions, percentage_contributions = calculate_portfolio_cvar(returns, weights, alpha)
    
    # Print results
    print(f"Portfolio CVaR: {portfolio_cvar:.6f}")
    print("CVaR Contributions by Asset:")
    for asset, contribution, percentage in zip(returns.columns, cvar_contributions, percentage_contributions):
        print(f"  {asset}: Contribution = {contribution:.6f}, Percentage = {percentage:.1%}")


Portfolio CVaR: 0.026764
CVaR Contributions by Asset:
  Asset 1: Contribution = 0.004100, Percentage = 15.3%
  Asset 2: Contribution = 0.021421, Percentage = 80.0%
  Asset 3: Contribution = 0.000561, Percentage = 2.1%
  Asset 4: Contribution = 0.000682, Percentage = 2.5%
